In [3]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('ec2-34-198-179-91.compute-1.amazonaws.com', 27017)
db = client.fletcher
dress_col = db.rtr_dresses
rev_col = db.rtr_reviews

In [4]:
dress_col.count()

11

In [5]:
rev_col.count()

10369

In [12]:
cur = rev_col.find()
df =  pd.DataFrame(list(cur))
del df['_id']

In [13]:
df.head(5)

,Age:,Body type:,Bust size:,Height:,Overall fit:,Rented for:,Size worn:,Usually wears:,Weight:,author,date,rating,review,title,url
0,NaN,NaN,NaN,NaN,NaN,Wedding,10R,NaN,NaN,RTR Customer,"February 14, 2017",reb-gold-stars-8,"Fits true to size. I'm 145 lb, 5'1"" and the 10...",Stunning dress,https://www.renttherunway.com/shop/designers/b...
1,28,Hourglass,NaN,"5' 9""",True to Size,Wedding,16R,14,200lbs,Lauran,"February 13, 2017",reb-gold-stars-10,I wish I could have gotten the 16L. I am 5'9 a...,Stunning dress! Super comfortable and I got lo...,https://www.renttherunway.com/shop/designers/b...
2,25,Pear,32B,"5' 1""",NaN,Formal Affair,2R,0,NaN,Anne,"February 12, 2017",reb-gold-stars-8,I loved this dress so much! I got lots of comp...,Lots of compliments all night!,https://www.renttherunway.com/shop/designers/b...
3,30,Athletic,34B,"5' 3""",True to Size,Formal Affair,6P,4,140lbs,Emilym,"February 12, 2017",reb-gold-stars-10,Great dress to highlight your waist with more ...,Comfortable and beautiful back detail,https://www.renttherunway.com/shop/designers/b...
4,NaN,NaN,NaN,NaN,NaN,Formal Affair,6R,NaN,NaN,Lisa,"February 12, 2017",reb-gold-stars-10,So many compliments on this dress. Would wear ...,GORGEOUS,https://www.renttherunway.com/shop/designers/b...


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10369 entries, 0 to 10368
Data columns (total 15 columns):
Age:               9480 non-null object
Body type:         7604 non-null object
Bust size:         7006 non-null object
Height:            7959 non-null object
Overall fit:       7365 non-null object
Rented for:        10369 non-null object
Size worn:         10369 non-null object
Usually wears:     7952 non-null object
Weight:            6502 non-null object
author             10369 non-null object
date               10369 non-null object
rating             10369 non-null object
review             10369 non-null object
title              10369 non-null object
url                10369 non-null object
dtypes: object(15)
memory usage: 1.2+ MB


In [11]:
cur_2 = dress_col.find()
dress_df =  pd.DataFrame(list(cur_2))
del dress_df['_id']

In [15]:
df.columns

Index(['Age: ', 'Body type: ', 'Bust size: ', 'Height: ', 'Overall fit: ',
       'Rented for: ', 'Size worn: ', 'Usually wears: ', 'Weight: ', 'author',
       'date', 'rating', 'review', 'title', 'url'],
      dtype='object')

In [20]:
new_cols = []
for col in df.columns:
    new_c = col.lower().strip().replace(' ', '_')
    new_c = new_c.replace(':', '')
    new_cols.append(new_c)

In [22]:
df.columns = new_cols

In [26]:
df.rating = df['rating'].str.extract(r'stars-([\d]+)', expand=True).astype(int)

In [37]:
df.loc[df.age.notnull(), 'age'] = df[df.age.notnull()].age.astype(int)

In [43]:
import numpy as np

In [44]:
df.loc[df.age == 1, 'age'] = np.nan

In [46]:
df.loc[df.age > 100, 'age'] = np.nan

## Text analysis
- right now i'm going to work only with text data
- i will explore the reviews first, and then the title.

In [49]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [99]:
#CountVectorizer is a class; so `vectorizer` below represents an instance of that object.
cv = CountVectorizer(ngram_range=(2, 2), stop_words="english")

# call `fit` to build the vocabulary
cv.fit(df.review)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [100]:
review_cv = cv.transform(df.review).toarray()

In [101]:
review_cv_df = pd.DataFrame(review_cv, columns = cv.get_feature_names())

In [102]:
count_sum = np.sum(review_cv, axis=0)

In [103]:
frequency_list = sorted(list(zip(cv.get_feature_names(), count_sum)), key = lambda x : x[1],reverse=True)

In [104]:
count_sum.shape

(106231,)

In [105]:
frequency_list[:20]

[('true size', 1058),
 ('loved dress', 996),
 ('dress fit', 908),
 ('fit perfectly', 790),
 ('received compliments', 725),
 ('got compliments', 717),
 ('compliments night', 647),
 ('fit great', 534),
 ('wore dress', 533),
 ('dress perfect', 470),
 ('ended wearing', 467),
 ('rose gold', 449),
 ('felt like', 424),
 ('great dress', 399),
 ('compliments dress', 386),
 ('dress comfortable', 367),
 ('highly recommend', 357),
 ('recommend dress', 357),
 ('like glove', 350),
 ('fit true', 349)]

In [107]:
from sklearn.decomposition import PCA
reducer = PCA(n_components = 20)
reduced_X = reducer.fit_transform(review_cv_df)

In [109]:
reduced_X.shape

(10369, 20)

In [110]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale

In [111]:
from sklearn.datasets import fetch_mldata
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

In [112]:
km = KMeans(n_clusters=20)
km.fit(reduced_X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [114]:
df['cv_kmeans'] = km.labels_

In [119]:
df[df.cv_kmeans == 1].review

1        I wish I could have gotten the 16L. I am 5'9 a...
3        Great dress to highlight your waist with more ...
4        So many compliments on this dress. Would wear ...
6        I am tall but the 8L was just a little too lon...
7        This dress is classic and extremely comfortabl...
15       Tight in the rib cage but worked out okay with...
18       This was a gorgeous dress -- I got the long ve...
21       This was the perfect dress for my friend's win...
25       I decided to go with the 12p based on length. ...
26       This dress hides "bumps" very well and I was i...
28       True to fit, length was perfect (I'm 5'8" plus...
32       5'3" 145lbs athletic build, 34D, wore 8R,  wit...
33       The dress came in with the side ripped and nee...
39       Had rented a different Badgley Mischka dress f...
40                                       Can't wear a bra 
41       My two original dresses became unavailable the...
42                      Beautiful dress! Very flattering